In [4]:
import lsst.daf.persistence as dafPersist
import lsst.afw.image as afwImage
import lsst.afw.table as afwTable
import lsst.afw.geom as afwGeom

butler = dafPersist.Butler("/home/boutigny/LSST/CFHT/clusters/MACSJ2243.3-0935/output/coadd_dir_cc")
dataid = {'tract':0, 'patch':"1,4", 'filter':'r'}

sources = butler.get('deepCoadd_meas', dataid)
 
deep = butler.get('deepCoadd_calexp', dataid)
#print "Found %d sources in this patch"%len(sources)

In [11]:
deepInfo = deep.getInfo().getCoaddInputs().ccds

In [13]:
ccdCalexp = {}
for ccd in deepInfo :
    ccdCalexp[ccd.getId()]=butler.get('calexp', ccd=ccd['ccd'], visit=ccd['visit'])

In [18]:
def getStamp(source) :
    allStamps = []
    for ccd in deepInfo :
        currentCalexp = ccdCalexp[ccd.getId()]
        if ccd.contains(source.getCoord()) :
            # get object footprint on the calexp
            fp = source.getFootprint().transform(deep.getWcs(), ccd.getWcs(), ccd.getBBox())
            if fp.getArea() != 0 :
                stamp = currentCalexp.Factory(currentCalexp, fp.getBBox(), afwImage.PARENT, True)
                allStamps.append(stamp)
    return allStamps

In [30]:
def getBBox(source) :
    count = 0
    maxWidth = 0
    maxHeight = 0
    for ccd in deepInfo :
        currentCalexp = ccdCalexp[ccd.getId()]
        if ccd.contains(source.getCoord()) :
            # get object footprint on the calexp
            fp = source.getFootprint().transform(deep.getWcs(), ccd.getWcs(), ccd.getBBox())
            if fp.getArea() != 0 :
                count += 1
                bbox = fp.getBBox()
                maxWidth = max(maxWidth, fp.getBBox().getWidth())
                maxHeight = max(maxHeight, fp.getBBox().getHeight())
    return count, maxWidth, maxHeight

In [33]:
allStamps = getStamp(sources[250])
count, maxWidth, maxHeight = getBBox(sources[250])
print len(allStamps), count, maxWidth, maxHeight

6 6 35 32


In [23]:
print dir(allStamps[1])

['Factory', '__class__', '__del__', '__delattr__', '__dict__', '__doc__', '__float__', '__format__', '__getattr__', '__getattribute__', '__getitem__', '__hash__', '__init__', '__int__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__swig_destroy__', '__swig_getmethods__', '__swig_setmethods__', '__weakref__', '_s', 'census', 'clone', 'getBBox', 'getCalib', 'getDetector', 'getDimensions', 'getFilter', 'getHeight', 'getId', 'getInfo', 'getMaskedImage', 'getMetadata', 'getNextMemId', 'getPsf', 'getWcs', 'getWidth', 'getX0', 'getXY0', 'getY0', 'hasBeenCorrupted', 'hasPsf', 'hasWcs', 'init', 'magicSentinel', 'markPersistent', 'readFits', 'repr', 'setCalib', 'setCorruptionCallback', 'setDeleteCallback', 'setDeleteCallbackId', 'setDetector', 'setFilter', 'setInfo', 'setMaskedImage', 'setMetadata', 'setNewCallback', 'setNewCallbackId', 'setPsf', 'setWcs', 'setXY0', 'swigConvert', 'this', 'writeFi

In [38]:
import lsst.afw.display.ds9 as ds9

wcs = allStamps[4].getWcs()
ds9.mtv(allStamps[4].getMaskedImage().getImage(),frame=2,wcs=wcs)